<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Software-Agents-Notebook-Q-learning-script-for-Auction-bot-that-optimally-purchases-an-item-at-a-maximum-value-during-an-auction." data-toc-modified-id="Software-Agents-Notebook-Q-learning-script-for-Auction-bot-that-optimally-purchases-an-item-at-a-maximum-value-during-an-auction.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Software Agents Notebook Q-learning script for Auction bot that optimally purchases an item at a maximum value during an auction.</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Analysis-function-below-for-performance" data-toc-modified-id="Analysis-function-below-for-performance-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Analysis function below for performance</a></span><ul class="toc-item"><li><span><a href="#Training-and-testing-block-for-learning-decay-occuring-only-at-random-action-only-step-." data-toc-modified-id="Training-and-testing-block-for-learning-decay-occuring-only-at-random-action-only-step-.-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Training and testing block for learning decay occuring only at random action only step .</a></span></li><li><span><a href="#Visualisation-block-for-graphing-results.-basic-Matplotlib-here." data-toc-modified-id="Visualisation-block-for-graphing-results.-basic-Matplotlib-here.-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Visualisation block for graphing results. basic Matplotlib here.</a></span></li></ul></li><li><span><a href="#Trial-Agent-where-constant-epsilon-decay-on-every-step." data-toc-modified-id="Trial-Agent-where-constant-epsilon-decay-on-every-step.-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Trial Agent where constant epsilon decay on every step.</a></span><ul class="toc-item"><li><span><a href="#Reinitialise-trial-agent-episilon" data-toc-modified-id="Reinitialise-trial-agent-episilon-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Reinitialise trial agent episilon</a></span></li><li><span><a href="#Training-and-testing-block-for-learning-decay-occuring-only-at-random-action-only-step-." data-toc-modified-id="Training-and-testing-block-for-learning-decay-occuring-only-at-random-action-only-step-.-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Training and testing block for learning decay occuring only at random action only step .</a></span></li><li><span><a href="#Visualisation-block-for-graphing-results.-basic-Matplotlib-here." data-toc-modified-id="Visualisation-block-for-graphing-results.-basic-Matplotlib-here.-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Visualisation block for graphing results. basic Matplotlib here.</a></span></li></ul></li></ul></li></ul></div>

# Software Agents Notebook Q-learning script for Auction bot that optimally purchases an item at a maximum value during an auction. 

## Introduction 

The purpose of this jupyter notebook is to outline how to initialise and appropriately train a software agent to bid appropriately for an item during an auction. 

In [ ]:
#Import libraries required for training 

import os 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns
import random

#Look at importing pyspark to make it go faster? 

In [ ]:
os.chdir(r'\\nsq024vs\u8\aczd087\MyDocs\Data science masters\Software Agents\Coursework\Results\SARSA New IDXMAX function')

In [ ]:
#Initialisation step define epsilon, alpha, gamma and value of agent. 
class Auction_Agent():
    
    import pandas as pd
    
    def __init__(self,duration_timesteps=0,epsilon=0,exploration_policy_decay=0,exploration_policy_decay_2=0,alpha=0,gamma=0,\
                 agent_value=0,max_bidding_price=0,No_States=0,No_Actions=0): #
        
        self.duration_timesteps=duration_timesteps
        self.epsilon=epsilon
        self.exploratory_decay=exploration_policy_decay
        self.exploratory_decay_2=exploration_policy_decay_2
        self.alpha=alpha
        self.gamma=gamma
        self.agent_value=agent_value
        self.Max_bidding_price=max_bidding_price
        self.No_States=int(self.Max_bidding_price+2) #Additional States added on for nan and exit states
        self.No_Actions=int(self.Max_bidding_price+2)
        self.zero_df=Auction_Agent.zero_df(self) 
               
    @classmethod
    def from_input(cls):
        return cls(
            int(input('No of Duration timesteps: ')), #Number of time steps in action. 
            float(input('Exploration policy, epsilon: ')),  # Rate at which exploratory learning occurs 
            float(input('Exploration policy decay, eps_decay: ')),  # Rate at which exploratory learning decays and exploitation policy takes over
            float(input('Exploration policy decay2, eps_decay2: ')),
            float(input('Learning rate, alpha: ')), #Learning rate update for Q matrix 
            float(input('Discount factor Parameter,gamma: ')), #Discount factor effect of future moves on decision matrix. 
            float(input('Agent auction value: ')), #Value of agent 
            float(input('Max bidding price: ')))
    
    @staticmethod #Static method to create attribute to be used in creating R and Q matrix methods 
    def zero_df(self): #self denotes that self attributes can be utilised here. 
        
        #Create dictionary to then be transformed into a dataframe which will be tranformed into a panel for 3-D reward matrix. 
        #Panel dictoinary will collect copies of final_dict and create a 3_d reward matrix. 

        final_dict={}
        
        zero_count=1 #initialising the size of zero list this will increase with time. 
        None_count=int(self.No_States-zero_count) #initialising the size of None list this will decrease with time.
        cols=1

        while cols < self.No_Actions: #While loop iteratethrough and creates dictionary to be used as T matrix.
            key=cols

            zero_list=[0]*zero_count #Create0 values for each column in dataframe
            none_list=[None]*None_count #Create NaN values for each column in dataframe

            final_list=zero_list+none_list #Combine values together to create column for keyvalue col

            zero_count+=1 #Iterate zero count and None count up and down respectivety in order to ensure diagonal matrix is 
            None_count-=1 #achieved

            if len(zero_list)==1: #Specific renaming of initial column as we start in NaN state to begin with in auction 
                final_dict['Nan']=final_list
            else:
                final_dict[cols-1]= final_list #Providing 

            cols+=1 #Increase column size up to total number of actions

        final_dict['Exit']=[0]*int(self.No_Actions)#Adding final reward matrix where 

        zero_df=pd.DataFrame.from_dict(final_dict) #Generates dataframe from dictionary
        zero_df.index=zero_df.columns #Convert dataframe index to be the same as columns for Nan and exit values
        
        return zero_df
        
    @property
    def R_matrix(self):
        #Creating final reward dataframe to iterate through next and add reward values 
        panel_dict={} 
        reward_df=self.zero_df.copy() #Copy used to stop dataframe acquiring new object name 

        R_mat_cols=list(reward_df.columns) #
        for cols in R_mat_cols: #removing all non number values 

            if isinstance(cols, str)==True:
                R_mat_cols.remove(cols)

        for vals in R_mat_cols: #Removing all zero values with reward values based on initial valuation set by user. 

            temp=reward_df[vals]
            reward_value=self.agent_value-vals
            temp.replace(0.0,reward_value,inplace=True)
            reward_df[vals]=temp

        for i in range(self.duration_timesteps): #Creating duplicated of zero matrices based on number of timesteps set by user
            panel_dict[i]=self.zero_df

        #print(panel_dict)
        panel_dict[self.duration_timesteps]=reward_df
        #print(panel_dict)
        return pd.Panel(panel_dict)
            
    
    @property #At property turns method 
    def Q_matrix(self): #self denotes that self attributes can be utilised here. 
        panel_dict={} 
        #Creating final reward dataframe to iterate through next and add reward values 
        
        for i in range(self.duration_timesteps+1): #Creating duplicated of zero matrices based on number of timesteps set by user
            panel_dict[i]=self.zero_df

        #print(panel_dict)
        return pd.Panel(panel_dict)
    


In [ ]:
trl=Qmatrix[t].iloc[state_index,state_index:]
count_zeros=trl[trl<0].count()
count_zeros
trl.count()-count_zeros

In [ ]:
trl[trl<0]

In [ ]:
trl

In [ ]:
def exploration_exploitation_rand_act_decay_policy_SARSA(Trial_agent,count_rand,count_arg_max,state_index):
    
   #Iterating through policy where random exploration or arg max exploitation decision is made"""
    
    if random.uniform(0, 1) < Trial_agent.epsilon:
        
        #Counting number of random actions taken per episode
        count_rand+=1
        #Select No states randomly for agent. 
        Action_index=random.randint(state_index, Trial_agent.No_States-1)
        #print(f'Random Action taken at time {t} for state {state_index} is {Action_index}')

        #Decay of exploration exploration policy based exploratory decay 
        if Trial_agent.epsilon>Trial_agent.exploratory_decay:
            Trial_agent.epsilon=Trial_agent.epsilon*Trial_agent.exploratory_decay_2
        else:
            Trial_agent.epsilon=Trial_agent.epsilon*0.001
            
    #Iterating through argmax policy if random policy not selected.       
    else:
        #Counting number of arg max actions taken per episode
        count_arg_max+=1
        
        temp=Qmatrix[t].iloc[state_index,state_index:]
        
        count_neg_vals=temp[temp<0].count()
        count_greater_zero=temp[temp>0].count()
        No_data_points=temp.count()
        Zero_max=No_data_points-count_neg_vals
        
        if count_greater_zero==0 and Zero_max==1:
            #print(Qmatrix[t].iloc[state_index,state_index:])
            Action_index=random.randint(state_index, Trial_agent.No_States-1)
            #print(f'Rand Argmax Action taken at time {t} for state {state_index} is {Action_index}')
        else:
            Non_num_index = Qmatrix[t].iloc[state_index,state_index:].idxmax() # Exploit learned values
            #print(Qmatrix[t].iloc[state_index,state_index:])
            Action_index=Columns.index(Non_num_index)
            #print(f'Non Rand Argmax Action taken at time {t} for state {state_index} is {Action_index}')
        
    return Action_index,count_rand,count_arg_max

In [ ]:
trl=pd.DataFrame(np.zeros(16).reshape(4,4))
trl.idxmax()

In [ ]:
def exploration_exploitation_constant_decay_policy_sarsa(Trial_agent,count_rand,count_arg_max,state_index):
    
   #Iterating through policy where random exploration or arg max exploitation decision is made"""
    
    if random.uniform(0, 1) < Trial_agent.epsilon:
        
        #Counting number of random actions taken per episode
        count_rand+=1
        #Select No states randomly for agent. 
        Action_index=random.randint(state_index, Trial_agent.No_States-1)
        #print(f'Random Action taken at time {t} for state {state_index} is {Action_index}')

    #Iterating through argmax policy if random policy not selected.       
    else:
        #Counting number of arg max actions taken per episode
        count_arg_max+=1

        Non_num_index = Qmatrix[t].iloc[state_index,state_index:].idxmax() # Exploit learned values

        Action_index=Columns.index(Non_num_index)
        #print(f'Argmax Action taken at time {t} for state {state_index} is {Action_index}')
        
        #Decay of exploration exploration policy based exploratory decay 
    if Trial_agent.epsilon>Trial_agent.exploratory_decay:
        Trial_agent.epsilon=Trial_agent.epsilon*Trial_agent.exploratory_decay_2
    else:
        Trial_agent.epsilon=Trial_agent.epsilon*0.001
    
    
    return Action_index,count_rand,count_arg_max

In [ ]:
def Generate_Q_update_values_SARSA(Rmatrix,Qmatrix,Trial_agent,Next_state_index,
                                   Next_Action_index,curr_state_index,curr_action_index):
    
    #Purpose of function to create new Q matrix value to update as per Bellman's equation. 

    Reward = Rmatrix[t].iloc[curr_state_index,curr_action_index] # Determine reward from taken action 
    #Creating parameters for iterating through for updating q matrix for analysis 
    #Acquire old q matrix value
    Q_old=Qmatrix[t].iloc[curr_state_index,curr_action_index]

    #Determine q matrix t+1 upper value provided not at t end point and agent has not selected exit action 
    if t<Trial_agent.duration_timesteps and Next_Action_index!=Trial_agent.No_States-1:
        
        Q_future_value=Qmatrix[t+1].iloc[Next_state_index,Next_Action_index]

    else:
        Q_future_value=0

    #Calculate Q update value
    Q_update=Q_old+Trial_agent.alpha*(Reward \
                                     +Trial_agent.gamma*Q_future_value
                                     -Q_old)
       
    return (Q_update, Reward)

In [ ]:
def Q_Matrix_converge_chk(Q_matrix_chk,
                          Q_matrix_temp,
                          Converge_count,
                          Converge_threshold,
                          Converge_breaker):
    """Purpose of this method is to check if Q matrix has converged. 
    This is determined my confirming that Q matrix has remained static for more than the convergence threshold"""

    
    if Q_matrix_chk.equals(Q_matrix_temp)==True: #If old Qmatrix equals new Q matrix update convergence count by 1 
        Converge_count+=1
    else: #If old Qmatrix does not equal new Q matrix reduce convergence count to 0 
        Converge_count=0
        
    if Converge_count==Converge_threshold:
        Converge_breaker=True
    else:
        Converge_breaker=False
    #print(f'method converge count is: ',Converge_count)
    #print(f'method converge breaker is: ',Converge_breaker)
    return Converge_count,Converge_breaker

## Analysis function below for performance

1. run Trial_agent below and input in duration of time steps episilon etc
2. Run Training and testing block. 
3. Run visualisation charts block to observe performance. 
4. If you want to re run the training and testing block you will need to reinitialise the trial agent episilon as it decays to 0 during training and testing analysis. 

In [ ]:
Trial_agent=Auction_Agent.from_input()


### Training and testing block for learning decay occuring only at random action only step . 

Grid Search on varying alpha and gamma values.

In [ ]:
#Grid Search on varying alpha and gamma values.

#------------------------GRID--SEARCH------------------------------------
avg_final_r = []
alphas = []
gammas = []
convpoints = []
random.seed(9001)
#grid search parameters
alphas_gs = [0.3,0.4,0.5,0.6, 0.7, 0.8, 0.9] #  0.1,0.2, 
gamma_gs = [0.1,0.2,0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]#  

for alpha in alphas_gs:
    
    for gamma in gamma_gs:
      
        #manually define variables
        #Trial_agent.duration_timesteps = 5
        Trial_agent.epsilon = 0.95
        Trial_agent.exploratory_decay = 0.2
        Trial_agent.alpha = alpha
        Trial_agent.gamma = gamma

        # testing and training block. 
        epoch_time = []
        epoch_reward = []
        epoch_no=[]
        exploration_decay=[]
        No_Explore_act=[]
        No_Exploit_act=[]
        # Number of epcohs performed for training and testing analysis. Increase as necessary., 
        #Determine threshold to switch from exploration to exploitation
        No_epochs_pt1=(Trial_agent.epsilon-Trial_agent.exploratory_decay)
        #print(No_epoch_pt1)
        No_epochs_pt2=(Trial_agent.epsilon\
                       -Trial_agent.epsilon*Trial_agent.exploratory_decay_2)
        #print(No_epochs_pt2)
        No_epochs_pt3=No_epochs_pt2/Trial_agent.epsilon
        #print(No_epochs_pt3)
        No_epochs_pt4=int(No_epochs_pt1/No_epochs_pt3)
        No_epochs=int(No_epochs_pt4+1.3*No_epochs_pt4)
        print(f'No of epochs: {No_epochs}')
        print('Starting epsilon: ',Trial_agent.epsilon)
        
        #Initialising blank R and Q matrices for training 
        
                #Convergence parameters
        converge_breaker=False
        Converge_count=0
        Conv_threshold=90
        
        filename='alpha_'+str(alpha)+'gamma_'+str(gamma)+'_SARSA.xlsx'
        conv_threshold_list=[]
        conv_count_list=[]
        Q_matrix_match=[]
        Q_sum_trend=[]
        Q_sum_abs_trend=[]
        Q_var_trend=[]
        Q_mean_trend=[]

        Qmatrix=Trial_agent.Q_matrix
        
        Rmatrix=Trial_agent.R_matrix

        #import ipdb
        #ipdb.sset_trace()

        #Iterate through each Epoch
        for i in range(1,No_epochs):
            #print(i)
            
            #Old Q matrix for comparison in convergence
            #Qmatrix_old=Qmatrix #Old Q matrix for comparison in convergence

            t=0 #Initialising time point for auction
            count_rand=0
            count_arg_max=0
            #Iterate through each time poin tin the 
            
            Qmatrix_old=Qmatrix.copy()
            Qmatrix_old_pt2=Qmatrix_old.values
            
            while t <= Trial_agent.duration_timesteps:
                #print(t)

                if t==0: #Initialising state for initial time step. 
                    state_index=0
                    #print(f'State index at time {t} is:{state_index}')
                    [Action_index, count_rand, count_arg_max]=exploration_exploitation_rand_act_decay_policy_SARSA(Trial_agent,
                                                                                                         count_rand,
                                                                                                         count_arg_max,
                                                                                                        state_index)
              
                
                    #print(f'Action index at time {t} is:{Action_index}')

                #Initialising q matrix columns index 
                Columns=list(Qmatrix[t].columns)
                
                New_state_index=Action_index
                #print(f'Updated state index at time {t} is: {New_state_index}')
                [New_Action_index, count_rand, count_arg_max]=exploration_exploitation_rand_act_decay_policy_SARSA(Trial_agent,
                                                                                                         count_rand,
                                                                                                         count_arg_max,
                                                                                                        New_state_index)
                #print(Trial_agent.epsilon)
             
                
                #print(f'Updated action index at time {t} is: {New_Action_index}')

                (Q_update,Reward)=Generate_Q_update_values_SARSA(Rmatrix,
                                                           Qmatrix, \
                                                           Trial_agent,\
                                                            New_state_index, \
                                                            New_Action_index,     
                                                           state_index,
                                                           Action_index)    

                #print(f'Q_update at time {t} value is {Q_update}')
                #print(f'Reward value at time {t} is {Reward}')

                Qmatrix[t].iloc[state_index,Action_index]=Q_update

                #Update for state index and move on to the next 

                #If Exit action is selected break whileloop and end training. 
                if New_state_index ==Trial_agent.No_States-1:
                    #print(f'Break action taken')
                    #Other actions to be written here 
                    break

                state_index=New_state_index
                
                Action_index=New_Action_index
                
                #print(f'New state index is {state_index}')
                
                t+=1 #Time point increases as each step in the auction improves
                #print(f'State index time {t} is :{state_index}')
                #print(f'Action index at time {t} is:{Action_index}')

            #Final results information. 
            epoch_time.append(t)
            epoch_reward.append(Reward)
            exploration_decay.append(Trial_agent.epsilon)
            epoch_no.append(i)
            No_Explore_act.append(count_rand)
            No_Exploit_act.append(count_arg_max)
            
            
            #Check convergence between old Q matrix and updated one. 
            (Converge_count,converge_breaker)=Q_Matrix_converge_chk(Qmatrix_old,\
                                                           Qmatrix,\
                                                           Converge_count,\
                                                           Conv_threshold,\
                                                          converge_breaker)
            
            conv_count_list.append(Converge_count)
            conv_threshold_list.append(converge_breaker)
            temp=Qmatrix.values
            temp2=Qmatrix.fillna(value=0)
            
            Q_matrix_match.append(np.array_equal(Qmatrix_old_pt2[~np.isnan(Qmatrix_old_pt2)], temp[~np.isnan(temp)]))
            Q_sum_abs_trend.append(Qmatrix.abs().sum().sum().sum())
            Q_sum_trend.append(Qmatrix.sum().sum().sum())
            Q_var_trend.append(temp2.var().var().var())
            Q_mean_trend.append(temp2.mean().mean().mean())
        
        

        results_df=pd.DataFrame(data={'Epoch_Number':epoch_no,\
                          'No_time_steps':epoch_time,\
                          'Final_reward':epoch_reward,\
                          'Exploration_decay':exploration_decay,\
                         'No_Explore_act':No_Explore_act,\
                         'No_Exploit_act':No_Exploit_act,\
                          'Convergence count': conv_count_list,\
                         'convergence Achieve': conv_threshold_list,\
                         'Q_matrix_match':Q_matrix_match,\
                         'Q_abs_sum_trend':Q_sum_abs_trend,\
                         'Q_sum_trend':Q_sum_trend,\
                         'Q_var_trend':Q_var_trend,\
                         'Q_mean_trend':Q_mean_trend})

        results_df['Rewards_moving_average']=results_df['Final_reward'].rolling(window=20,center=False).mean()
        results_df['No_steps_taken_moving_average']=results_df['No_time_steps'].rolling(window=20,center=False).mean()
        results_df['No_Explore_act_moving_average']=results_df['No_Explore_act'].rolling(window=20,center=False).mean()
        results_df['No_Exploit_act_moving_average']=results_df['No_Exploit_act'].rolling(window=20,center=False).mean()
            
        #Convergence point   
        #find convergence point (point with most significant negative parcentage change)
        convpoint = results_df[results_df['Exploration_decay'].pct_change() < -0.2].Epoch_Number.min()
        results_df.to_excel(filename)
        #Average final reward
        #isolate final 100 epochs 
        results_f100 = results_df.iloc[-100:, :]
        #avgerage the reward of final 100 epochs
        avg_final_reward = results_f100['Final_reward'].mean()
        
        #print prompts for checks/processing status
        print('Learning Rate (alpha): ',Trial_agent.alpha)
        print('Discount Factor (gamma) : ',Trial_agent.gamma) 
        print('Final epsilon: ',Trial_agent.epsilon)
        print("Avg final reward: ", avg_final_reward)
        print("Convergence Point: ", convpoint)
        print(" ")
        
        #append grid search results into respective lists
        avg_final_r.append(avg_final_reward)
        gammas.append(gamma)
        alphas.append(alpha)
        convpoints.append(convpoint)

#compile grid search results lists into dataframe
grid_df=pd.DataFrame(data={'Alpha':alphas,\
                           'Gamma':gammas,\
                           'Avg_Final_reward':avg_final_r,\
                           'Convergence Epoch':convpoints})


## Store and collate final reuslts from each run into a overall excel spreadsheet and store plot as heatmap

In [ ]:
import re

In [ ]:
os.chdir(r'\\nsq024vs\u8\aczd087\MyDocs\Data science masters\Software Agents\Coursework\Results\SARSA results')

files=os.listdir()


In [ ]:
#Analyse and assess the files run results. 

Final_results=pd.DataFrame(columns=['Alpha','Gamma','Converg_count','Avg_500Final_reward'])

for f in files:
    print(f)
    data = pd.read_excel(f)
    temp_alpha=re.search("alpha_", f).start()
    temp_gamma=re.search("gamma_", f).start()
    
    alpha_val=float(f[temp_alpha+6:temp_alpha+9])
    gamma_val=float(f[temp_gamma+6:temp_gamma+9])
    
    Converged_idx=0
    Final_reward=0
    
    for idx,vals in data['Convergence count'].iteritems():
        if vals ==500:
            Converged_idx=idx
    
    Final_reward=data['Final_reward'][-100:].mean()
    
    Final_results.loc[len(Final_results)] = [alpha_val,gamma_val,Converged_idx,Final_reward]


In [ ]:
grid_df_2 = Final_results.pivot("Alpha", "Gamma", "Avg_500Final_reward")
sns_plot =sns.heatmap(grid_df_2, cmap="Greens")
fig = sns_plot.get_figure()
fig.savefig("Alpha_gamma_avrg_reward_SARSA.png")

In [ ]:
grid_df_3 = Final_results.pivot("Alpha", "Gamma", "Converg_count")

sns_plot =sns.heatmap(grid_df_3, cmap="Blues")
fig = sns_plot.get_figure()
fig.savefig("Alpha_gamma_steps_convergence_SARSA.png")

## Get per Epoch trends

In [ ]:
Final_results=pd.DataFrame()
Final_results

#Analyse and assess the files run results. 

files =os.listdir()

Final_results=pd.DataFrame()

for f in files:
    #print(f)
    data = pd.read_excel(f)
    temp_alpha=re.search("alpha_", f).start()
    temp_gamma=re.search("gamma_", f).start()
    
    alpha_val=f[temp_alpha+6:temp_alpha+9]
    gamma_val=f[temp_gamma+6:temp_gamma+9]
    col_name='alpha_'+alpha_val+'gamma_'+gamma_val
    
    
    for cols in data.columns:
        
        if cols=='Q_abs_sum_trend':
            
            temp=data['Q_abs_sum_trend']
            temp.rename(col_name)

            Final_results[col_name]=temp
            
        
    Final_results
    #Final_reward=data['Final_reward'][Converged_idx:Converged_idx+100].mean()
    
    #Final_results.loc[len(Final_results)] = [alpha_val,gamma_val,Converged_idx,Final_reward]

    

In [ ]:
#Produce  figures for performance
selected_columns= ['alpha_0.1gamma_0.1','alpha_0.1gamma_0.5','alpha_0.1gamma_0.9',\
          'alpha_0.5gamma_0.1','alpha_0.5gamma_0.5','alpha_0.5gamma_0.9',\
        'alpha_0.9gamma_0.1','alpha_0.9gamma_0.5','alpha_0.9gamma_0.9']

font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 15}

plt.rc('font', **font)

ax=Final_results.loc[:,selected_columns].plot(figsize=(20, 10),legend=False)

patches, labels = ax.get_legend_handles_labels()

ax.legend(patches, labels, loc='right')

ax.set_ylabel('Q table absolute sum value per epoch')
ax.set_xlabel('Number of Epochs')
plt.savefig('SARSA_abs_value_vs_No_epochs.jpeg')
plt.show()


## Get per Epoch trends

In [ ]:
Final_results=pd.DataFrame()
Final_results

#Analyse and assess the files run results. 

files =os.listdir()

Final_results=pd.DataFrame()

for f in files:
    #print(f)
    data = pd.read_excel(f)
    temp_alpha=re.search("alpha_", f).start()
    temp_gamma=re.search("gamma_", f).start()
    
    alpha_val=f[temp_alpha+6:temp_alpha+9]
    gamma_val=f[temp_gamma+6:temp_gamma+9]
    col_name='alpha_'+alpha_val+'gamma_'+gamma_val
    
    
    for cols in data.columns:
        
        if cols=='Q_abs_sum_trend':
            
            temp=data['Q_abs_sum_trend']
            temp.rename(col_name)

            Final_results[col_name]=temp
            
        
    Final_results
    #Final_reward=data['Final_reward'][Converged_idx:Converged_idx+100].mean()
    
    #Final_results.loc[len(Final_results)] = [alpha_val,gamma_val,Converged_idx,Final_reward]

    

In [ ]:
#Produce  figures for performance
selected_columns= ['alpha_0.1gamma_0.1','alpha_0.1gamma_0.5','alpha_0.1gamma_0.9',\
          'alpha_0.5gamma_0.1','alpha_0.5gamma_0.5','alpha_0.5gamma_0.9',\
        'alpha_0.9gamma_0.1','alpha_0.9gamma_0.5','alpha_0.9gamma_0.9']

font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 15}

plt.rc('font', **font)

ax=Final_results.loc[:,selected_columns].plot(figsize=(20, 10),legend=False)

patches, labels = ax.get_legend_handles_labels()

ax.legend(patches, labels, loc='right')

ax.set_ylabel('Q table absolute sum value per epoch')
ax.set_xlabel('Number of Epochs')
plt.savefig('SARSA_abs_value_vs_No_epochs.jpeg')
plt.show()


### Visualisation block for graphing results. basic Matplotlib here. 

In [ ]:
#results_df.set_index(epoch_no,inplace=True)
results_df.plot(subplots=True,figsize=(20, 15))
plt.legend(loc='best')

In [ ]:
#Plot of final reward vs final reward moving average. 
results_df.plot(y=['Final_reward','Rewards_moving_average'],figsize=(20, 20))
plt.legend(loc='best')
plt.xlabel('No of episodes')

In [ ]:
#Plot of final reward vs final reward moving average. 
results_df.plot(y=['No_time_steps','No_steps_taken_moving_average'],figsize=(20, 20))
plt.legend(loc='best')
plt.xlabel('No of episodes')

## Trial Agent where constant epsilon decay on every step. 

### Reinitialise trial agent episilon

In [ ]:
Trial_agent.epsilon=0.95
Trial_agent.epsilon

### Training and testing block for learning decay occuring only at random action only step . 

In [ ]:
# testing and training block. 
epoch_time = []
epoch_reward = []
epoch_no=[]
exploration_decay=[]
No_Explore_act=[]
No_Exploit_act=[]
No_epochs=1000 # Number of epcohs performed for training and testing analysis. Increase as necessary., 
#Initialising blank R and Q matrices for training 

Qmatrix=Trial_agent.Q_matrix 
Rmatrix=Trial_agent.R_matrix

#import ipdb
#ipdb.sset_trace()

#Iterate through each Epoch
for i in range(1,No_epochs):
    #print(i)

    t=0 #Initialising time point for auction
    count_rand=0
    count_arg_max=0
    #Iterate through each time poin tin the 
    while t <= Trial_agent.duration_timesteps:
        #print(t)


        if t==0: #Initialising state for initial time step. 
            state_index=0
        
        #Initialising q matrix columns index 
        Columns=list(Qmatrix[t].columns)

        [Action_index, count_rand, count_arg_max]=exploration_exploitation_constant_decay_policy(Trial_agent,count_rand,count_arg_max)

        (Q_update,Reward)=Generate_Q_update_values(Rmatrix,Qmatrix,Trial_agent,state_index,Action_index)    

        #print(f'Q_update value is {Q_update}')

        Qmatrix[t].iloc[state_index,Action_index]=Q_update

        #Update for state index and move on to the next 
             
        #If Exit action is selected break whileloop and end training. 
        if state_index ==Trial_agent.No_States-1:
            #print(f'Break action taken')
            #Other actions to be written here 
            break
            
        state_index=Action_index
        #print(f'New state index is {state_index}')

        t+=1 #Time point increases as each step in the auction improves
    
    
    #Final results information. 
    epoch_time.append(t)
    epoch_reward.append(Reward)
    exploration_decay.append(Trial_agent.epsilon)
    epoch_no.append(i)
    No_Explore_act.append(count_rand)
    No_Exploit_act.append(count_arg_max)


### Visualisation block for graphing results. basic Matplotlib here. 

In [ ]:
results_df=pd.DataFrame(data={'Epoch_Number':epoch_no,\
                              'No_time_steps':epoch_time,\
                              'Final_reward':epoch_reward,\
                              'Exploration_decay':exploration_decay,\
                             'No_Explore_act':No_Explore_act,\
                             'No_Exploit_act':No_Exploit_act})
results_df['Rewards_moving_average']=results_df['Final_reward'].rolling(window=20,center=False).mean()
results_df['No_steps_taken_moving_average']=results_df['No_time_steps'].rolling(window=20,center=False).mean()
results_df['No_Explore_act_moving_average']=results_df['No_Explore_act'].rolling(window=20,center=False).mean()
results_df['No_Exploit_act_moving_average']=results_df['No_Exploit_act'].rolling(window=20,center=False).mean()
#results_df.set_index(epoch_no,inplace=True)
results_df.plot(subplots=True,figsize=(20, 15))
plt.legend(loc='best')

In [ ]:
#Plot of final reward vs final reward moving average. 
results_df.plot(y=['Final_reward','Rewards_moving_average'],figsize=(20, 20))
plt.legend(loc='best')
plt.xlabel('No of episodes')

In [ ]:
#Plot of final reward vs final reward moving average. 
results_df.plot(y=['No_time_steps','No_steps_taken_moving_average'],figsize=(20, 20))
plt.legend(loc='best')
plt.xlabel('No of episodes')